In [3]:
import pandas as pd


stopwords = pd.read_csv('../data/stopwords.txt', names=['stopword'], header=None)
stopwords = stopwords['stopword']
stopwords = stopwords.str.replace('.', r'\.', regex=False)

clients = pd.read_csv('../data/clientes.csv', usecols=['nombre'])
clients['trimmed_name'] = clients['nombre']

for word in stopwords:
    clients['trimmed_name'] = clients['trimmed_name'].str.replace(r'(?:\s|^){}(?:\s|$)'.format(word), ' ', regex=True).str.strip()

clients.to_csv('../data/clean_names.csv', index=False)

In [4]:
import pandas as pd
import regex


def match_and_extract(row, *, text_col):
    pattern = r'\b(?<=(.{0,100}))('+'(.{0,20})'.join(row['trimmed_name'].split())+r')\b(?=(.{0,100}))'
#     pattern = r'\b(?<=(.{0,100}))('+row['trimmed_name']+r')\b(?=(.{0,100}))'
#     print(row['trimmed_name'])
    matches = regex.findall(pattern, row[text_col], flags=regex.I|regex.DOTALL)
#     print(matches)
    matches = [' '.join(x) for x in matches]
    return matches

regex_chars = ['.','(', ')']
client_df = pd.read_csv('../data/clientes.csv')
regroup_desc_ciiu_division = pd.read_csv('../data/regroup_desc_ciiu_division.csv')
client_df = pd.merge(client_df, regroup_desc_ciiu_division, how='left', on='desc_ciiu_division')
client_df['group'] = 'group_' + client_df['group'].astype(str)
clean_names = pd.read_csv('../data/clean_names.csv')
client_df = client_df.merge(clean_names, on='nombre')
for char in regex_chars:
    client_df['trimmed_name'] = client_df['trimmed_name'].str.replace(char, f'\{char}', regex=False)


client_news_df = pd.read_csv('../data/clientes_noticias.csv')[['nit', 'news_url_absolute','news_id']]
news_df = pd.read_csv('../data/noticias.csv')


client_news_df = client_news_df.merge(news_df, on='news_id')
client_news_df = client_news_df.merge(client_df, on='nit')

client_news_df['appearance_in_title'] = client_news_df.apply(match_and_extract, text_col='news_title', axis=1)
client_news_df['appearance_in_body'] = client_news_df.apply(match_and_extract, text_col='news_text_content', axis=1)

client_news_df['name_in_title'] = client_news_df['appearance_in_title'].map(len) > 0
client_news_df['name_in_body'] = client_news_df['appearance_in_body'].map(len) > 0

client_news_df['appearance_in_title'] = client_news_df['appearance_in_title'].map(str)
client_news_df['appearance_in_body'] = client_news_df['appearance_in_body'].map(str)

# print(client_news_df[client_news_df[['name_in_body', 'name_in_body']].any(axis=1)])

client_news_df[client_news_df[['name_in_body', 'name_in_body']].any(axis=1)].drop(columns=['news_title', 'news_text_content', 'trimmed_name']).to_csv('../data/text_matching.csv', index=False)

In [5]:
get_rid_list = ['NIVEL',
               'SI',
               'PROTECCION',
               'EFICACIA',
               'CADENA',
               'P R',
               'CONTINENTE',
               'ACTIVOS',
               'EFECTIVO',
               'MINEROS',
               'ETERNA',
               'MAYORISTA',
               'ADQUIRIR',
               'DIVISA',
                'SURAMERICANA',
                'LISTOS',
                'SERVICIOS PUBLICOS',
                'ULTRA'
               ]

In [6]:
client_news_df[((client_news_df.name_in_body==True) | (client_news_df.name_in_title==True)) & (~client_news_df.trimmed_name.isin(get_rid_list))].trimmed_name.value_counts()

UNIVERSIDAD LOS ANDES                              82
BANCO INTERAMERICANO DESARROLLO BID                79
HOSPITAL UNIVERSITARIO                             78
SOBERANA                                           62
METRO BOGOTA                                       61
                                                   ..
ENERGIA BOYACA                                      1
CAJA COMPENSACION FAMILIAR CAMPESINA COMCAJA        1
CAJA COMPENSACION FAMILIAR COMFENALCO ANTIOQUIA     1
ECODIESEL COLOMBIA                                  1
DUMIAN MEDICAL                                      1
Name: trimmed_name, Length: 416, dtype: int64

In [7]:
matched_news = client_news_df[((client_news_df.name_in_body==True) | (client_news_df.name_in_title==True) | \
                               ((client_news_df.news_id == 'news23736') & (client_news_df.nit == 800047031))) & \
                              (~client_news_df.trimmed_name.isin(get_rid_list))]
matched_news.shape

(3626, 20)

In [8]:
matched_news = pd.merge(matched_news, matched_news.groupby('news_id').nit.count().to_frame().rename(columns={'nit':'count'}), on='news_id')

In [9]:
matched_news = matched_news[matched_news['count'] < 2]
matched_news

,nit,news_url_absolute_x,news_id,news_url_absolute_y,news_init_date,news_final_date,news_title,news_text_content,nombre,desc_ciiu_division,...,desc_ciiuu_clase,subsec,group,desc_ciiu_division_eng,trimmed_name,appearance_in_title,appearance_in_body,name_in_title,name_in_body,count
22,860034313,https://www.semana.com/mejor-colombia/articulo...,news10333,https://www.semana.com/mejor-colombia/articulo...,2022-07-30,2022-08-14,Un seguro contra el cambio climatico. Asi se b...,Uno de los grandes desafios del sector agricol...,BANCO DAVIVIENDA SA,"ACTIVIDADES DE SERVICIOS FINANCIEROS, EXCEPTO ...",...,BANCOS COMERCIALES,BANCOS,group_30,"FINANCIAL SERVICES ACTIVITIES, EXCEPT INSURANC...",BANCO DAVIVIENDA,[],"['ilidad a los colombianos "", afirma Alvaro Ca...",False,True,1
23,860034313,https://www.semana.com/economia/capsulas/artic...,news10341,https://www.semana.com/economia/capsulas/artic...,2022-07-30,2022-08-14,Banco Davivienda aumento su inversion indirect...,Copyright (c) 2022 Publicaciones Semana S.A NI...,BANCO DAVIVIENDA SA,"ACTIVIDADES DE SERVICIOS FINANCIEROS, EXCEPTO ...",...,BANCOS COMERCIALES,BANCOS,group_30,"FINANCIAL SERVICES ACTIVITIES, EXCEPT INSURANC...",BANCO DAVIVIENDA,[' Banco Davivienda aumento su inversion in...,[],True,False,1
24,860034313,https://www.iproup.com/finanzas/32808-rappi-pe...,news10348,https://www.iproup.com/finanzas/32808-rappi-pe...,2022-07-15,2022-07-30,"Rappi, PedidosYa: que servicios financieros of...","La competencia por convertirse en la nueva ""su...",BANCO DAVIVIENDA SA,"ACTIVIDADES DE SERVICIOS FINANCIEROS, EXCEPTO ...",...,BANCOS COMERCIALES,BANCOS,group_30,"FINANCIAL SERVICES ACTIVITIES, EXCEPT INSURANC...",BANCO DAVIVIENDA,[],['esta disponible en los siguientes mercados :...,False,True,1
25,860034313,https://www.laopinion.com.co/economia/los-banc...,news10351,https://www.laopinion.com.co/economia/los-banc...,2022-07-15,2022-07-30,Los bancos que le permiten abrir cuentas en do...,En medio de la coyuntura por el alza del dolar...,BANCO DAVIVIENDA SA,"ACTIVIDADES DE SERVICIOS FINANCIEROS, EXCEPTO ...",...,BANCOS COMERCIALES,BANCOS,group_30,"FINANCIAL SERVICES ACTIVITIES, EXCEPT INSURANC...",BANCO DAVIVIENDA,[],['l exterior para hacerlo. Algunas de las enti...,False,True,1
26,860034313,https://www.elheraldo.co/judicial/policia-frus...,news10354,https://www.elheraldo.co/judicial/policia-frus...,2022-07-15,2022-07-30,Policia frustra 'taquillazo' en entidad bancar...,Dos delincuentes fueron capturados por la Poli...,BANCO DAVIVIENDA SA,"ACTIVIDADES DE SERVICIOS FINANCIEROS, EXCEPTO ...",...,BANCOS COMERCIALES,BANCOS,group_30,"FINANCIAL SERVICES ACTIVITIES, EXCEPT INSURANC...",BANCO DAVIVIENDA,[],"[""turados por la Policia, en una rapida reacci...",False,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3621,890212568,https://consultorsalud.com/fcv-certificacion-e...,news25630,https://consultorsalud.com/fcv-certificacion-e...,2022-07-15,2022-07-30,FCV recibe certificacion EMRAM 7 en salud digital,"En dias anteriores, durante el Himss Executive...",FUNDACION CARDIOVASCULAR DE COLOMBIA FCV,ACTIVIDADES DE ATENCION DE LA SALUD HUMANA,...,"ACTIVIDADES DE HOSPITALES Y CLINICAS, CON INTE...",IPS,group_41,HUMAN HEALTH CARE ACTIVITIES,FUNDACION CARDIOVASCULAR COLOMBIA FCV,[],"['En dias anteriores, durante el Himss Executi...",False,True,1
3622,800216499,https://www.levante-emv.com/el-mercantil-valen...,news46163,https://www.levante-emv.com/el-mercantil-valen...,2022-07-30,2022-08-14,Los nuevos amos del ladrillo,La inmobiliaria con mas peso en Valencia y su ...,AGOFER SAS,COMERCIO AL POR MENOR (INCLUSO EL COMERCIO AL ...,...,COMERCIO AL POR MENOR DE ARTICULOS DE FERRETER...,FERRETERIA,group_22,RETAIL TRADE (INCLUDING RETAIL TRADE OF FUELS)...,AGOFER,[],"[', creo la firma en 2007 con parte de los fon...",False,True,1
3623,800216499,https://www.levante-emv.com/el-mercantil-valen...,news46175,https://www.levante-emv.com/el-mercantil-valen...,2022-07-15,2022-07-3

In [10]:
# def get_top_occurence(row, *, column_name):
#     return row[column_name].value_counts().idxmax()   

# matched_news_top_occurence_des_ciiu_division = matched_news.groupby('news_id').apply(get_top_occurence, column_name='desc_ciiu_division')

# matched_news_top_occurence_des_ciiu_division = pd.merge(matched_news_top_occurence_des_ciiu_division.to_frame(), matched_news[['news_id', 'news_title','news_text_content']].drop_duplicates(), on='news_id')
# matched_news_top_occurence_des_ciiu_division.rename(columns={0:'des_ciiu_division'}, inplace=True)
# matched_news_top_occurence_des_ciiu_division

In [11]:
client_df[~client_df.group.isin(matched_news.group.unique())].group.unique()

array(['group_3'], dtype=object)

In [12]:
matched_news.group.value_counts()

group_40    739
group_30    553
group_1     196
group_20    161
group_23    140
group_44    124
group_39    109
group_21    107
group_18     96
group_41     68
group_11     67
group_43     63
group_49     57
group_22     54
group_13     54
group_28     54
group_33     49
group_2      32
group_25     30
group_19     27
group_36     25
group_26     21
group_31     19
group_12     17
group_27     13
group_29     12
group_16     10
group_46      9
group_14      7
group_5       7
group_48      5
group_4       5
group_32      4
group_6       2
group_38      2
group_47      1
group_35      1
group_7       1
group_42      1
Name: group, dtype: int64

In [13]:
matched_news.to_csv('matched_news_group.csv', index=False)